In [ ]:
!pip install -qq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00


# dataset for transcribe

In [ ]:
!pip install -qq --upgrade datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.3 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# If the dataset is gated/private, make sure you have run huggingface-cli login
dataset = load_dataset("common_voice",'th',split="test")

In [ ]:
dataset = dataset.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

# Set up (pipline use gpu)

In [ ]:
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, Pipeline
import torch

class MyWav2Vec2Pipeline(Pipeline):# Chnage to your model on huggingface
    def __init__(self, model="Pongsathorn/wav2vec2_56000", processor="Pongsathorn/wav2vec2_56000", device=0):
        self.model = Wav2Vec2ForCTC.from_pretrained(model).to("cuda" if torch.cuda.is_available() else "cpu")
        self.processor = Wav2Vec2Processor.from_pretrained(processor)
        super().__init__(model=self.model, tokenizer=self.processor, device=device)

    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs):
        waveform, original_sampling_rate = torchaudio.load(inputs)
        resampler = torchaudio.transforms.Resample(orig_freq=original_sampling_rate, new_freq=16000)
        resampled_array = resampler(waveform).numpy().flatten()

        input_values = self.processor(resampled_array, sampling_rate=16_000, return_tensors="pt").input_values
        return {"input_values": input_values.to(self.device)}

    def _forward(self, model_inputs):
        logits = self.model(model_inputs["input_values"]).logits
        return {"logits": logits}

    def postprocess(self, model_outputs):
        predicted_ids = torch.argmax(model_outputs["logits"], dim=-1)
        transcription = self.processor.batch_decode(predicted_ids)[0]
        return {"transcription": transcription}


In [ ]:
from transformers.pipelines import PIPELINE_REGISTRY

PIPELINE_REGISTRY.register_pipeline(
    "wav2vec2",
    pipeline_class=MyWav2Vec2Pipeline,
)


In [ ]:
my_pipeline = MyWav2Vec2Pipeline()

In [ ]:
dataset

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 2188
})

In [ ]:
from tqdm import tqdm

transcriptions = []

# Moving pipeline computations to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
my_pipeline.model.to(device)

for idx in tqdm(range(len(dataset))):
    audio_path = dataset[idx]['path']
    try:
        # You should move your inputs to the same device as your model
        result = my_pipeline(audio_path)
        transcriptions.append(result['transcription'])
    except Exception as e:
        print(f"Error processing audio file {audio_path}: {str(e)}")
        transcriptions.append(None)  # or some error value



  0%|          | 10/2188 [00:16<15:21,  2.36it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 2188/2188 [03:06<00:00, 11.72it/s]


In [ ]:
transcriptions[0]

'เรา เริ่มต้น ด้วย วิธี นี้'

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({
    'transcriptions': transcriptions
})

In [ ]:
df

,transcriptions
0,เรา เริ่มต้น ด้วย วิธี นี้
1,ส่ง ให้ ตรง นี้ แหละ โดโด้ บอก
2,หันทาง ที่ ดี ที่สุด ไป สู่ ความ มุ่งหวัง ของ ...
3,คุณค่ง ว่าง มาก เลย สิ นะ
4,พรุ่ง นี้ เขา จะ มา ทำ ถนน หน้า บ้าน นะ อาจ จ...
...,...
2183,แอนดไผ่ ดิน เคย เป็น ที่ นิโยอ ตั้น มาก สำหรั...
2184,แก้วแตก และ คลองเหล ใน นั้น หก
2185,คา หนุ่ม เงย น่า จัด หนังสือ เมื่อ เห็น ว่า ไม...
2186,ฉัน สามารถ ทำ อไร ให้ คุณ ได้ บ้าง


In [ ]:
df.to_csv('transcribe.csv')

# prepare text

In [ ]:
import pandas as pd

df = pd.read_csv('/content/transcribe.csv')

In [ ]:
! pip install -qq pythainlp
!pip install -qq deepcut

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 93.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.2 MB/s eta 0:00:00


In [ ]:
from pythainlp.tokenize import word_tokenize
import re

In [ ]:
import gc
from tqdm.auto import tqdm
import pandas as pd

def clean_data(data):
    pattern = r"[^ก-๙a-zA-Z0-9\s]+"
    cleaned_data = re.sub(pattern, '', data)
    return cleaned_data

def th_words_tokenize(sentence):
    sentence = clean_data(sentence)  # apply the cleaning function
    tokens = word_tokenize(sentence.replace('เเ', 'แ'), engine="deepcut")
    tokens = [tokens[i-1] if token == 'ๆ' and i > 0 else token for i, token in enumerate(tokens)]
    return " ".join(tokens).replace("  ", " ")




In [ ]:
dataset['sentence'][0]

'เราเริ่มต้นด้วยวิธีนี้'

In [ ]:
from tqdm import tqdm

batch_size = 1000
preprocessed_dataset = []

for i in tqdm(range(0, len(dataset), batch_size)):
    # get the current batch
    batch = dataset['sentence'][i:i+batch_size]
    for data in batch:
        # tokenize the sentence
        preprocessed_data = th_words_tokenize(data)
        preprocessed_dataset.append(preprocessed_data)


# To dataframe

In [ ]:
preprocessed_dataset[0:10]

['เรา เริ่มต้น ด้วย วิธี นี้',
 'ส่ง ให้ ตรง นี้ แหละ  โดโด้ บอก',
 'หนทาง ที่ ดี ที่สุด ไป สู่ ความ มุ่ง หวัง ของ ฉัน คือ อะไร',
 'คุณคง ว่าง มาก เลย สิ นะ',
 'พรุ่ง นี้ เขา จะ มา ทำ ถนน หน้า บ้าน นะ  อาจ จะ เสียง ดัง หน่อย',
 'ฉัน พินิจ ใน จินตนาการ ของ ฉัน',
 'เรา มา จาก จังหวัดระยอง',
 'เคลวินฟาเรนไฮต์ หรือ เซลเซียส ซึ่ง เป็น หน่วย อุณหภูมิ ที่ ดี ที่สุด',
 'การ แข่งขัน เป็น อย่าง ไร บ้าง',
 'และ มัน ก็ เป็น ความ จริง']

In [ ]:
tokenized_df = pd.DataFrame({
    'tokenized_sentences': [data for data in preprocessed_dataset],
})

In [ ]:
tokenized_df.head()

,tokenized_sentences
0,เรา เริ่มต้น ด้วย วิธี นี้
1,ส่ง ให้ ตรง นี้ แหละ โดโด้ บอก
2,หนทาง ที่ ดี ที่สุด ไป สู่ ความ มุ่ง หวัง ของ ...
3,คุณคง ว่าง มาก เลย สิ นะ
4,พรุ่ง นี้ เขา จะ มา ทำ ถนน หน้า บ้าน นะ อาจ จ...


In [ ]:
df.head()

,Unnamed: 0,transcriptions
0,0,เรา เริ่มต้น ด้วย วิธี นี้
1,1,ส่ง ให้ ตรง นี้ แหละ โดโด้ บอก
2,2,หันทาง ที่ ดี ที่สุด ไป สู่ ความ มุ่งหวัง ของ ...
3,3,คุณค่ง ว่าง มาก เลย สิ นะ
4,4,พรุ่ง นี้ เขา จะ มา ทำ ถนน หน้า บ้าน นะ อาจ จ...


In [ ]:
df_eval = pd.concat([tokenized_df, df], axis=1)

In [ ]:
df_eval.sample(20)

,tokenized_sentences,Unnamed: 0,transcriptions
358,อย่า ทำ เสียง ดัง นะ,358,อย่า ทำ เสียง ดัง นะ
1618,กระตุก เชือก และ ส่งทอง ออก ไป,1618,กระตุก เชือก และ ส่ง ทอง ออก ไป
1507,คุณ ต้อง ตอบ ว่า มี ความ สุข ดี,1507,คุณ ต้อง ตอบ ว่า มี ความ สุข ดี
840,แบบ นี้ จะ ท้อง ไหม,840,แบบ นี้ จะ ท้อง ไหม
161,แต่ ทรัพยากร ของ ฉัน กำลัง จะ หมด,161,แต่ ทรัพยากร ของ ฉัน กำลัง จะ หมด
156,ที่ รัก มัน คือ คำ ถาม ที่ อยู่ ใน ใจ ของ คุณ,156,ที่ รัก มัน คือ คำ ถาม ที่ อยู่ ใน ใจ ของ คุณ
464,ฉัน กำลัง มี อาการ บาดเจ็บ ที่ ข้อ ต่อ,464,ฉัน กำลัง มี อาการ บาดเจ็บ ที่ ข้อ ต่อ
2073,การ บัญชี เป็น ชื่อ ของ อาชีพ สำหรับ นัก บัญชี,2073,การ บัญชี นึ ชื่อ ของ มาชีพ สุบานัก บัญชี
1982,ป้า ของ ฉัน กำลัง ทำ งาน,1982,ตา ของ ฉัน กำลัง ทำ งาน
585,เรา จะ ออก นอก เมือง มาก สุด ใน เดือน พฤษภาคม,585,เรา จะ ออก นอก เมือง มาก สุด ใน เดือน พฤษภาคม


In [ ]:
!pip install -qq jiwer
!pip install -qq evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.1 MB/s eta 0:00:00


In [ ]:
import evaluate

In [ ]:
wer_metric = evaluate.load("wer")

In [ ]:
wer_metric.compute(predictions=df_eval['transcriptions'], references=df_eval["tokenized_sentences"])

0.16027721247456136

In [ ]:
cer_metric = evaluate.load("cer")

In [ ]:
df_eval['tokenized_sentences']= df_eval['tokenized_sentences'].apply(lambda x : x.replace(' ',''))
df_eval['transcriptions']= df_eval['transcriptions'].apply(lambda x : x.replace(' ',''))

In [ ]:
cer_metric.compute(predictions=df_eval.transcriptions,references=df_eval.tokenized_sentences)

0.0478111936832327